# My attempt at Case Study 1:

In [1]:
# Play a card game in which the cards are iteratively flipped until we tell the dealer to stop, then one additional card is flipped over
# Find a strategy to make the last card flipped over a red card
# I have already seen this before, so will try a strategy I think I saw before

In [4]:
# Make The Deck:
red_cards = 26 * [1]
black_cards = 26 * [0]
full_deck = red_cards + black_cards

In [5]:
# Get all permutations with NumPy:
import numpy as np
np.random.seed(0)
shuffled_deck = np.random.permutation(full_deck)

In [6]:
print(shuffled_deck)
print(type(shuffled_deck))

[0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 1
 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0]
<class 'numpy.ndarray'>


In [1]:
# Play the game where we will keep track of the number of black and red cards remaining in the deck
# and stop the game when the number of red cards outnumbers the number of black cards:
# this function will return 1 if the game is won and 0 if the game is lost
def play_game(shuffled_deck):  
    red_total = 26
    black_total = 26
    final_card = shuffled_deck[0]
    for index, card in np.ndenumerate(shuffled_deck):
        if card == 1:
            red_total -= 1
        else:
            black_total -= 1

        if red_total > black_total:
            # halt the game because there are more red cards than black cards
            final_card = shuffled_deck[index[0] + 1]
            print("Halted")
            break
            
        # record the last card in case there are never more red cards than black cards
        final_card = card
    
    print("Final Check")
    if final_card == 1:
        return 1
    else:
        return 0

In [6]:
print(play_game(shuffled_deck))

0


In [20]:
from collections import defaultdict


In [22]:
np.random.seed(0)
# counters for the wins and losses
num_wins = 0
num_losses = 0

# play 100 games with a random deck and record it
for _ in range(100):
    shuffled_deck = np.random.permutation(full_deck)
    # print(shuffled_deck)
    if play_game(shuffled_deck) == 1:
        num_wins += 1
    else:
        num_losses += 1
        
print(num_wins)
print(num_losses)

58
42


# Case Study 1 Solution

In [1]:
# 4.1 Predicting Red Cards in a Shuffled Deck
red_cards = 26 * [1]
black_cards = 26
total_cards = red_cards + black_cards

In [5]:
import numpy as np
# Shuffle the deck
np.random.seed(1)
shuffled_deck = np.random.permutation(total_cards)

In [9]:
# Terminate the game when the number of red cards is greater than the number of black cards
remaining_red_cards = 26
for i, card in enumerate(shuffled_deck[:-1]):
    remaining_red_cards -= card
    remaining_total_cards = 52 - i - 1
    if (remaining_red_cards / remaining_total_cards) > 0.5:
        break
        
print(f"Stopping the game at index {i}.")
final_card = shuffled_deck[i + 1]
color = 'red' if final_card else 0
print(f"The next card in the deck is {'red' if final_card else 'black'}.")
print(f"We have {'won' if final_card else 'lost'}!")

Stopping the game at index 50.
The next card in the deck is red.
We have won!


In [11]:
# Generalizing the Card Game Strategy
np.random.seed(0)
total_cards = 52
total_red_cards = 26
def execute_strategy(min_fraction_red=0.5, shuffled_deck=None, return_index=False):
    if shuffled_deck is None:
        shuffled_deck = np.random.permutation(unshuffled_deck)
        
    remaining_red_cards = total_red_cards
    
    for i, card in enumerate(shuffled_deck[:-1]):
        remaining_red_cards -= card
        fraction_red_cards = remaining_red_cards / (total_cards - i - 1)
        if fraction_red_cards > min_fraction_red:
            break
        
    # returns either the return index and the final card as a tuple: (return_index, final card) or just the final card
    # depending on the return_index input parameter
    return (i+1, shuffled_deck[i+1]) if return_index else shuffled_deck[i+1]

In [ ]:
# 4.1.1 Estimating the probability of strategy success